# 如何在 Web 环境中使用 LangGraph.js

LangGraph.js 使用 [`async_hooks`](https://nodejs.org/api/async_hooks.html)
API 可以更方便地允许在内部进行跟踪和回调传播
节点。许多环境都支持此 API，例如
[Node.js](https://nodejs.org/api/async_hooks.html),
[Deno](https://deno.land/std@0.177.0/node/internal/async_hooks.ts),
[Cloudflare Workers](https://developers.cloudflare.com/workers/runtime-apis/nodejs/asynclocalstorage/)，
和
[Edge 运行时](https://vercel.com/docs/functions/runtimes/edge-runtime#兼容-node.js-modules)，
但不是全部，例如在网络浏览器中。

允许在没有的环境中使用 LangGraph.js
`async_hooks` API 可用，我们添加了单独的 `@langchain/langgraph/web`
入口点。该入口点导出主入口点的所有内容
`@langchain/langgraph` 导出，但不会初始化，甚至不会导入
[[代码4]]。这是一个简单的例子：

In [1]:
// 从“@langchain/langgraph/web”导入
import {
  END,
  START,
  StateGraph,
  Annotation,
} from "@langchain/langgraph/web";
import { BaseMessage, HumanMessage } from "@langchain/core/messages";

const GraphState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

const nodeFn = async (_state: typeof GraphState.State) => {
  return { messages: [new HumanMessage("Hello from the browser!")] };
};

// 定义一个新图
const workflow = new StateGraph(GraphState)
  .addNode("node", nodeFn)
  .addEdge(START, "node")
  .addEdge("node", END);

const app = workflow.compile({});

// 使用可运行的
const finalState = await app.invoke(
  { messages: [] },
);

console.log(finalState.messages[finalState.messages.length - 1].content);

Hello from the browser!


其他入口点，例如 `@langchain/langgraph/prebuilt`，可用于
无论是环境。

<div class="警告警告">
<p class="admonition-title">警告</p>
<p>
如果您在前端使用 LangGraph.js，请确保您没有暴露任何私钥！
对于聊天模型，这意味着您需要使用类似 <a href="https://js.langchain.com/docs/integrations/chat/web_llm">WebLLM</a>
无需身份验证即可运行客户端。
</p>
</div>

## 传递配置

网络浏览器缺乏 `async_hooks` 支持意味着如果您正在调用
一个 [`Runnable`](https://js.langchain.com/docs/concepts/runnables/) 内
节点（例如调用聊天模型时），需要手动传入一个
`config` 对象通过正确支持跟踪，
[`.streamEvents()`](https://js.langchain.com/docs/how_to/streaming#using-stream-events)
流式传输中间步骤和其他回调相关功能。这
配置对象将作为每个节点的第二个参数传入，并且应该是
用作任何 `Runnable` 方法的第二个参数。

为了说明这一点，让我们像以前一样再次设置我们的图表，但是使用
`Runnable` 在我们的节点内。首先，我们将避免将 `config` 传递到
嵌套函数，然后尝试使用 `.streamEvents()` 查看中间
嵌套函数的结果：

In [2]:
// 从“@langchain/langgraph/web”导入
import {
  END,
  START,
  StateGraph,
  Annotation,
} from "@langchain/langgraph/web";
import { BaseMessage } from "@langchain/core/messages";
import { RunnableLambda } from "@langchain/core/runnables";
import { type StreamEvent } from "@langchain/core/tracers/log_stream";

const GraphState2 = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

const nodeFn2 = async (_state: typeof GraphState2.State) => {
  // 请注意，我们不会在此处传递任何“config”
  const nestedFn = RunnableLambda.from(async (input: string) => {
    return new HumanMessage(`Hello from ${input}!`);
  }).withConfig({ runName: "nested" });
  const responseMessage = await nestedFn.invoke("a nested function");
  return { messages: [responseMessage] };
};

// 定义一个新图
const workflow2 = new StateGraph(GraphState2)
  .addNode("node", nodeFn2)
  .addEdge(START, "node")
  .addEdge("node", END);

const app2 = workflow2.compile({});

// 从图中流式传输中间步骤
const eventStream2 = app2.streamEvents(
  { messages: [] },
  { version: "v2" },
  { includeNames: ["nested"] },
);

const events2: StreamEvent[] = [];
for await (const event of eventStream2) {
  console.log(event);
  events2.push(event);
}

console.log(`Received ${events2.length} events from the nested function`);

Received 0 events from the nested function


我们可以看到我们没有收到任何事件。

接下来，让我们尝试使用传递配置的节点重新声明图形
正确：

In [3]:
// 从“@langchain/langgraph/web”导入
import {
  END,
  START,
  StateGraph,
  Annotation,
} from "@langchain/langgraph/web";
import { BaseMessage } from "@langchain/core/messages";
import { type RunnableConfig, RunnableLambda } from "@langchain/core/runnables";
import { type StreamEvent } from "@langchain/core/tracers/log_stream";

const GraphState3 = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

// 请注意这里的第二个参数。
const nodeFn3 = async (_state: typeof GraphState3.State, config?: RunnableConfig) => {
  // 如果需要嵌套更深，记得调用时传递 `_config`
  const nestedFn = RunnableLambda.from(
    async (input: string, _config?: RunnableConfig) => {
      return new HumanMessage(`Hello from ${input}!`);
    },
  ).withConfig({ runName: "nested" });
  const responseMessage = await nestedFn.invoke("a nested function", config);
  return { messages: [responseMessage] };
};

// 定义一个新图
const workflow3 = new StateGraph(GraphState3)
  .addNode("node", nodeFn3)
  .addEdge(START, "node")
  .addEdge("node", END);

const app3 = workflow3.compile({});

// 从图中流式传输中间步骤
const eventStream3 = app3.streamEvents(
  { messages: [] },
  { version: "v2" },
  { includeNames: ["nested"] },
);

const events3: StreamEvent[] = [];
for await (const event of eventStream3) {
  console.log(event);
  events3.push(event);
}

console.log(`Received ${events3.length} events from the nested function`);

{
  event: "on_chain_start",
  data: { input: { messages: [] } },
  name: "nested",
  tags: [],
  run_id: "22747451-a2fa-447b-b62f-9da19a539b2f",
  metadata: {
    langgraph_step: 1,
    langgraph_node: "node",
    langgraph_triggers: [ "start:node" ],
    langgraph_task_idx: 0,
    __pregel_resuming: false,
    checkpoint_id: "1ef62793-f065-6840-fffe-cdfb4cbb1248",
    checkpoint_ns: "node"
  }
}
{
  event: "on_chain_end",
  data: {
    output: HumanMessage {
      "content": "Hello from a nested function!",
      "additional_kwargs": {},
      "response_metadata": {}
    }
  },
  run_id: "22747451-a2fa-447b-b62f-9da19a539b2f",
  name: "nested",
  tags: [],
  metadata: {
    langgraph_step: 1,
    langgraph_node: "node",
    langgraph_triggers: [ "start:node" ],
    langgraph_task_idx: 0,
    __pregel_resuming: false,
    checkpoint_id: "1ef62793-f065-6840-fffe-cdfb4cbb1248",
    checkpoint_ns: "node"
  }
}
Received 2 events from the nested function


您可以看到我们按预期从嵌套函数中获取了事件。

## 后续步骤

您现在已经了解了使用 LangGraph.js 的一些特殊注意事项
在网络环境中。

接下来，检查一下
[一些关于核心功能的操作指南](/langgraphjs/how-tos/#core)。